In [1]:
from functools import reduce
import pandas as pd
from random import sample

In [2]:
# IRA: Indicador de recarga de acuifero, rango [1, 5]
# W: Porderadores de las variables (Resultado de procesar encuestas expertos), rango [, ]
# V: Valores de las variables para un lugar dado, rango [, ]
def IRA(W, V):
    return sum(w * v for (w, v) in zip(W, V))

In [3]:
# Genera nombres para columans e indices
def var_list(prefix, n):
    return [prefix + str(x + 1) for x in range(n)]    

In [4]:
# Genera resultado encuesta de un experto para n variables
def expert_survey(n):
    return sample(range(1, n+1), n)

In [5]:
# Genera resultados de encuesta de m expertos para n varibles 
def data_survey(n=13, m=10):
    # Genera un DataFrame vacio
    df = pd.DataFrame(pd.np.nan, index=var_list('v', n), columns=var_list('expert', m))
    # Rellena encuesta con resultados aleatorios
    for column in df:
        df[column] = expert_survey(n)
    return df

In [6]:
df = data_survey(13, 10)
df

,expert1,expert2,expert3,expert4,expert5,expert6,expert7,expert8,expert9,expert10
v1,6,8,7,13,13,5,10,7,9,11
v2,5,12,6,9,5,6,9,9,2,3
v3,8,10,8,7,3,4,1,10,7,2
v4,1,6,3,2,8,12,4,11,4,12
v5,9,9,13,11,10,9,12,1,11,9
v6,4,3,11,12,2,2,8,8,6,6
v7,12,1,10,4,6,10,2,6,3,5
v8,13,4,9,6,12,13,13,4,1,8
v9,2,13,5,3,9,3,3,2,12,13
v10,11,11,2,5,11,7,6,3,8,4


In [7]:
# Guarda encuesta en un excel
df.to_excel('data.xlsx')
# Leer excel guardado
# pd.read_excel('data.xlsx')

In [8]:
def diff(x, y):
    return x - y

def Fc(df):
    aux = pd.DataFrame()
    for index, row in df.iterrows():
        k = df.apply(lambda row: diff(row, df.loc[index,:]), axis=1)
        l = k.mask(k > 0, 1).where(k > 0, 0)
        aux[index] = l.apply(sum)
    return aux.T

In [9]:
Fc(df)

,expert1,expert2,expert3,expert4,expert5,expert6,expert7,expert8,expert9,expert10
v1,7,5,6,0,0,8,3,6,4,2
v2,8,1,7,4,8,7,4,4,11,10
v3,5,3,5,6,10,9,12,3,6,11
v4,12,7,10,11,5,1,9,2,9,1
v5,4,4,0,2,3,4,1,12,2,4
v6,9,10,2,1,11,11,5,5,7,7
v7,1,12,3,9,7,3,11,7,10,8
v8,0,9,4,7,1,0,0,9,12,5
v9,11,0,8,10,4,10,10,11,1,0
v10,2,2,11,8,2,6,7,10,5,9


In [10]:
mult = lambda x, y: x * y

def Wt(df):
    return df.apply(sum, axis=1) / reduce(mult, df.shape)

In [11]:
# Calcula pesos relativos
Wt_ = Wt(Fc(df))

In [12]:
# Calcula ponderaciones
W = Wt_ / sum(Wt_)
W

v1     0.052564
v2     0.082051
v3     0.089744
v4     0.085897
v5     0.046154
v6     0.087179
v7     0.091026
v8     0.060256
v9     0.083333
v10    0.079487
v11    0.067949
v12    0.083333
v13    0.091026
dtype: float64

In [13]:
sum(W)

1.0

In [14]:
# Guardar a Excel
W.to_excel('W.xlsx', header=['W'])
# Leer desde Excel
# pd.read_excel('W.xlsx')